# Project Vision: Human Image Segmentation and Skeleton Extraction
## Team Members: Francisco Garcia and Melanie Pacheco

We aim to tackle the task of human image segmentation and skeleton extraction. Our goal is to develop an algorithm that can segment human shapes in images, creating masks, and then extract the skeleton of the important body parts.

### Overview
In this project, we are focusing on the critical task of understanding human anatomy from images. This involves two main steps:

1. **Human Image Segmentation**: We aim to develop a model that can accurately identify and segment human shapes within images. This process involves creating masks that delineate the boundaries of the human body.

2. **Skeleton Extraction**: Once we have segmented the human shape, our next objective is to extract the skeleton of the significant body parts. This skeleton provides a structural representation of the human body, highlighting key joints and limbs.

### How to run it using main.py

In this file you can see the step by step implementation and explanation of the steps. But if you want to run it in terminal you can use

```python:
python main.py
```
In the main.py file, pass the path of the image to ```img_path``` variable, and set the ```display_img``` variable to True for diplaying each step.

## Pipeline Explanation

**Library import and setup**

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from scipy.ndimage import distance_transform_edt
from skimage.morphology import skeletonize
from skimage import data, segmentation, color
from skimage import graph
from scipy.ndimage import convolve
from scipy.spatial.distance import pdist, squareform
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import minimum_spanning_tree
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from matplotlib import pyplot
from numpy import unique
from numpy import where

### Human Image Segmentation

In [3]:
def display_image(img,  title='Image', gray=False):
    # Display
    plt.imshow(img)

    # plot if the image is gray
    if gray:
        plt.gray()

    plt.title(title)
    plt.axis('off')
    plt.show()

### Skeleton Extration